In [1]:
import pandas as pd, numpy as np

# Add 'industry oil' to coverage input data

In [2]:
## pull in (previously final) input data sheet
outputs_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/'
input_data_path = outputs_dir + 'lsff_input_coverage_data.csv'

input_data = pd.read_csv(input_data_path)

loc_id_map = {i:j for(i,j) in zip(input_data.location_name, input_data.location_id) }

In [3]:
# pull in (previously final) version of input data sheet with metadata
w_meta = pd.read_csv(outputs_dir + 'lsff_input_coverage_data_with_metadata.csv')

In [4]:
# pull in new numbers from Jonathan / industry
inputs_dir = "/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/"
new_data = pd.read_excel(inputs_dir + "20210411_LSFF_input_data_for_IHME.xlsx", header = [1,2,3])

In [5]:
# subset to the oil numbers we're interested in
new_data = new_data.set_index(new_data.columns[0])

new_oil = new_data['Baseline coverage']['Oil\n(percent of population eating specified oil)'][['Fortifiable oil.1','Compliance % of fortifiable']]

In [6]:
# pull columns we're interested in
rename_oil = {
    ('Location', 'Unnamed: 0_level_1', 'Unnamed: 0_level_2'): 'location_name',
 'Fortifiable oil.1': 'percent of population eating industrially produced vehicle',
 'Compliance % of fortifiable': 'percent of population eating fortified vehicle'
}

new_oil = new_oil.reset_index().rename(columns=rename_oil)

In [7]:
# reshape to our format
new_oil = pd.melt(new_oil, id_vars = 'location_name',
        value_vars = ['percent of population eating industrially produced vehicle',
       'percent of population eating fortified vehicle'],
       var_name='value_description',value_name='value_mean')

In [8]:
# fix c'ote divoire 
new_oil.loc[new_oil.location_name=="CÃ´te d'Ivoire","location_name"] = "Côte d'Ivoire"

In [9]:
# make sure we're not missing any locations / don't have any extra
assert(set(new_oil.location_name).symmetric_difference(set(input_data.location_name))==set())

In [10]:
# formatting
new_oil['vehicle'] = 'industry oil'
new_oil['nutrient'] = 'vitamin a'
new_oil['u5_applicable'] = True
new_oil['wra_applicable'] = True
new_oil['sub_population'] = np.NaN
new_oil['location_id'] = new_oil.location_name.map(loc_id_map)

In [11]:
# check formatting
new_oil.head()

,location_name,value_description,value_mean,vehicle,nutrient,u5_applicable,wra_applicable,sub_population,location_id
0,Angola,percent of population eating industrially prod...,72.320,industry oil,vitamin a,True,True,NaN,168
1,Bangladesh,percent of population eating industrially prod...,87.516,industry oil,vitamin a,True,True,NaN,161
2,Burkina Faso,percent of population eating industrially prod...,73.760,industry oil,vitamin a,True,True,NaN,201
3,Côte d'Ivoire,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,205
4,Cameroon,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,202


In [12]:
# from our (previously final) input sheet, select extracted rows of data (no regression or multiplicative results)
# then drop any rows duplicated for different nutrients
w_meta = w_meta[w_meta.estimation_status=='na'].drop(columns = 'nutrient').drop_duplicates()

In [13]:
# use ratio of scale over mean to estimate scale for new industry datapoints
w_meta['scale_over_mean'] = (w_meta.value_975_percentile - w_meta.value_025_percentile) / w_meta.value_mean
r = w_meta.loc[(w_meta.scale_over_mean!=np.inf) & (w_meta.scale_over_mean!=0),['vehicle','scale_over_mean']]

r_mean = r.scale_over_mean.mean()
r.loc[r.vehicle=="maize flour",'scale_over_mean'] = r_mean

r = r.groupby('vehicle').mean().dropna().rename(columns={'scale_over_mean':'r'}).reset_index()

In [14]:
r

,vehicle,r
0,maize flour,0.564982
1,oil,0.379944
2,wheat flour,0.688639


In [15]:
# we'll use the oil-specific ratio
new_oil['r'] = 0.379944

In [16]:
new_oil.head()

,location_name,value_description,value_mean,vehicle,nutrient,u5_applicable,wra_applicable,sub_population,location_id,r
0,Angola,percent of population eating industrially prod...,72.320,industry oil,vitamin a,True,True,NaN,168,0.379944
1,Bangladesh,percent of population eating industrially prod...,87.516,industry oil,vitamin a,True,True,NaN,161,0.379944
2,Burkina Faso,percent of population eating industrially prod...,73.760,industry oil,vitamin a,True,True,NaN,201,0.379944
3,Côte d'Ivoire,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,205,0.379944
4,Cameroon,percent of population eating industrially prod...,72.000,industry oil,vitamin a,True,True,NaN,202,0.379944


In [18]:
# input estimated CIs
new_oil['value_025_percentile'] = np.clip(new_oil.value_mean - (new_oil.r * new_oil.value_mean)/2, 0, 100)
new_oil['value_975_percentile'] = np.clip(new_oil.value_mean + (new_oil.r * new_oil.value_mean)/2, 0, 100)

In [19]:
output = input_data.append(new_oil)

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v108/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


In [20]:
output.tail()

,location_id,location_name,nutrient,r,sub_population,u5_applicable,value_025_percentile,value_975_percentile,value_description,value_mean,vehicle,wra_applicable
45,196,South Africa,vitamin a,0.379944,NaN,True,2.430084,3.569916,percent of population eating fortified vehicle,3.0,industry oil,True
46,522,Sudan,vitamin a,0.379944,NaN,True,0.000000,0.000000,percent of population eating fortified vehicle,0.0,industry oil,True
47,190,Uganda,vitamin a,0.379944,NaN,True,44.065523,64.734477,percent of population eating fortified vehicle,54.4,industry oil,True
48,189,United Republic of Tanzania,vitamin a,0.379944,NaN,True,43.417501,63.782499,percent of population eating fortified vehicle,53.6,industry oil,True
49,20,Viet Nam,vitamin a,0.379944,NaN,True,56.701960,83.298040,percent of population eating fortified vehicle,70.0,industry oil,True


In [21]:
output.columns

Index(['location_id', 'location_name', 'nutrient', 'r', 'sub_population',
       'u5_applicable', 'value_025_percentile', 'value_975_percentile',
       'value_description', 'value_mean', 'vehicle', 'wra_applicable'],
      dtype='object')

In [ ]:
output.loc[(output.value_mean >= output.value_975_percentile) & (output.value_description!='percent of population eating vehicle')]

In [ ]:
output.loc[(output.value_mean <= output.value_025_percentile) & (output.value_description!='percent of population eating vehicle')]

In [ ]:
output.to_csv('/ihme/homes/beatrixh/repos/scratch/lsff_input_coverage_data.csv')